<a href="https://colab.research.google.com/github/weathon/3d2smile/blob/main/CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install rdkit deepsmiles

In [ ]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses huggingface_hub

In [ ]:
!pip install transformers -U

In [ ]:
!wget http://file.weasoft.com/80k.csv

In [1]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
import deepsmiles
import numpy as np
import pylab
converter = deepsmiles.Converter(rings=True, branches=True)
def deepsmiles_to_img(ds):
   img = np.array(Draw.MolToImage(Chem.MolFromSmiles(converter.decode(ds)), size=(400,400)).convert("L", dither=None).convert("RGB"))
   img = np.where(img<253, 0, 1) * img
   return img

def smiles_to_img(smiles):
  return deepsmiles_to_img(converter.encode(smiles))

In [2]:
import pandas
csv = pandas.read_csv("80k.csv")

In [3]:
# prompt: convert all the data in canonicalsmiles column using converter.encode function and store them in a new array

smiles_arr = []
for smiles in csv['canonicalsmiles']:
  smiles_arr.append(converter.encode(smiles))


In [4]:
import torch
import torchvision

In [5]:
image_encoder = torchvision.models.efficientnet_v2_l(weights="DEFAULT")

In [51]:
smiles_encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-larget-uncased')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


In [52]:
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-large-cased')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


In [79]:
a = smiles_encoder(torch.tensor([tokenizer.encode("I am playing a PC", add_special_tokens=True)])).last_hidden_state[:,0,:]

In [83]:
b = smiles_encoder(torch.tensor([tokenizer.encode("I hate playing on a PC", add_special_tokens=True)])).last_hidden_state[:,0,:]

In [78]:
c = smiles_encoder(torch.tensor([tokenizer.encode("I am playing a computer", add_special_tokens=True)])).last_hidden_state[:,0,:]

In [75]:
cs = torch.nn.CosineSimilarity()

In [84]:
cs(a,b)

tensor([0.6361], grad_fn=<SumBackward1>)

In [85]:
cs(a,c)

tensor([0.9435], grad_fn=<SumBackward1>)